# Gold Report - SQL Version

This notebook creates reporting tables using DLT SQL syntax.

In [ ]:
import dlt
from pyspark.sql.functions import *

In [ ]:
@dlt.table(
    name="gold_report_sql",
    comment="Detailed reporting metrics per category - SQL version"
)
def gold_report_sql():
    gold_df = dlt.read("gold_table")
    
    # Calculate total sales across all categories
    total_sales = gold_df.agg(sum("total_sales")).collect()[0][0]
    
    # Add percentage and performance category
    result = gold_df.withColumn(
        "sales_percentage",
        round((col("total_sales") / lit(total_sales) * 100), 2)
    ).withColumn(
        "performance_category",
        when(col("total_sales") > lit(total_sales) * 0.25, "High Performer")
        .when(col("total_sales") > lit(total_sales) * 0.15, "Medium Performer")
        .otherwise("Low Performer")
    )
    
    return result

In [ ]:
@dlt.table(
    name="executive_summary",
    comment="Executive summary with overall business metrics"
)
def executive_summary():
    gold_df = dlt.read("gold_table")
    
    summary = gold_df.agg(
        sum("total_sales").alias("total_revenue"),
        sum("total_transactions").alias("total_transactions_count"),
        round(avg("avg_transaction_value"), 2).alias("overall_avg_transaction_value"),
        sum("total_quantity_sold").alias("total_items_sold"),
        countDistinct("category").alias("total_categories"),
        min("first_transaction_date").alias("business_start_date"),
        max("last_transaction_date").alias("latest_transaction_date")
    )
    
    return summary

In [ ]:
@dlt.table(
    name="category_ranking",
    comment="Category performance rankings"
)
def category_ranking():
    from pyspark.sql.window import Window
    
    gold_df = dlt.read("gold_table")
    
    # Create window spec for ranking
    window_spec = Window.orderBy(col("total_sales").desc())
    
    ranked = gold_df.withColumn(
        "sales_rank",
        rank().over(window_spec)
    ).withColumn(
        "transaction_rank",
        rank().over(Window.orderBy(col("total_transactions").desc()))
    ).select(
        "category",
        "sales_rank",
        "transaction_rank",
        "total_sales",
        "total_transactions",
        "avg_transaction_value"
    )
    
    return ranked

## Verification

Run these queries after pipeline completes:

```sql
SELECT * FROM main.demo.gold_report_sql;
SELECT * FROM main.demo.executive_summary;
SELECT * FROM main.demo.category_ranking;
```